# TensorFlow - Tensorboard

- #tensorboard

references: https://github.com/golbin/TensorFlow-Tutorials

텐서플로를 사용해서 사이즈가 큰 신경망 네트워크를 계산하다보면, 계산과정이 복잡하고 헷갈리기 마련입니다. 텐서보드는 이런 복잡한 계산과정을 조금이라도 이해하기 쉽게 하기 위한 도구입니다. 내부적으로 그래프, 이미지, 히스토그램 등 함수와 변수가 어떻게 변해가는지를 쉽게 볼수 있습니다.

![텐서보드 예제](./images/TensorboardCNN1.png)

## 텐서보드 실행

```
tensorboard --logdir=./logs
http://localhost:6006
```

## 레이어

레이어는 다음과 같이 설정합니다.

``` py
with tf.name_scope('layer'):
    w1 = tf.Variable(tf.random_uniform([784,100], -1, 1), name='w1')
    h1 = tf.nn.relu(tf.matmul(x, w1))
    
    tf.summary.histogram("x", x)
```

한 레이어 안에 w1과 h1를 설정한 모습입니다. 텐서보드 안에서 w1과 h1은 한 레이어 안에 표현됩니다.

tf.summary 안에 메소드를 사용하여 얻고싶은 값들을 지정할수 있습니다. 레이어 안에 같이 지정해줘야 합니다.

### 스칼라

``` py
tf.summary.scalar("x", x)
```

### 히스토그램

``` py
tf.summary.histogram("x", x)
```

## 그래프 설정 및 표시하고싶은 텐서 설정
``` py
# 스칼라나 히스토그램으로 보여주려는 변수들 모음
merged = tf.summary.merge_all()
# 그래프와 텐서값 저장공간 설정
writer = tf.summary.FileWriter('./logs', sess.graph)
```

## 값을 얻고 저장
``` py
writer.add_summary(summary, global_step=sess.run(global_step))
```

# MNIST 숫자필체 판별 예제

https://www.tensorflow.org/tutorials/layers

In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.examples.tutorials.mnist import input_data
from tensorflow.python.client import device_lib

# Import Dataset
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)
train_data = mnist.train
eval_data = mnist.test

# Check Device
device_lib.list_local_devices()

c:\git\venv\deepnn\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 14772794299259862195, name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 5586505728
 locality {
   bus_id: 1
 }
 incarnation: 4966557896555366174
 physical_device_desc: "device: 0, name: GeForce GTX 1060 6GB, pci bus id: 0000:01:00.0, compute capability: 6.1"]

In [2]:
# Build Model
global_step = tf.Variable(-1, trainable=False, name='global_step')
increment_global_step_op = tf.assign(global_step, global_step+1)

num_filters1 = 32
num_filters2 = 64
num_units1 = 7*7*num_filters2
num_units2 = 1024

with tf.name_scope('input'):
    x = tf.placeholder(tf.float32, [None, 784], name='input')
    x_image = tf.reshape(x, [-1,28,28,1])

with tf.name_scope('conv1'):
    W_conv1 = tf.Variable(tf.truncated_normal([5,5,1,num_filters1], stddev=0.1), name = 'kernel1')
    h_conv1 = tf.nn.conv2d(x_image, W_conv1, strides=[1,1,1,1], padding='SAME')
    b_conv1 = tf.Variable(tf.constant(0.1, shape=[num_filters1]), name = 'conv_bias1')
    h_conv1_cutoff = tf.nn.relu(h_conv1 + b_conv1)
    h_pool1 = tf.nn.max_pool(h_conv1_cutoff, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')

with tf.name_scope('conv2'):
    W_conv2 = tf.Variable(tf.truncated_normal([5,5,num_filters1,num_filters2], stddev=0.1), name = 'kernel2')
    h_conv2 = tf.nn.conv2d(h_pool1, W_conv2, strides=[1,1,1,1], padding='SAME')
    b_conv2 = tf.Variable(tf.constant(0.1, shape=[num_filters2]), name = 'conv_bias2')
    h_conv2_cutoff = tf.nn.relu(h_conv2 + b_conv2)
    h_pool2 = tf.nn.max_pool(h_conv2_cutoff, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')
    h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*num_filters2])

with tf.name_scope('dense1'):
    w2 = tf.Variable(tf.truncated_normal([num_units1, num_units2]), name='weight1')
    b2 = tf.Variable(tf.constant(0.1, shape=[num_units2]), name = 'dense_bias1')
    hidden2 = tf.nn.relu(tf.matmul(h_pool2_flat, w2) + b2)
    keep_prob = tf.placeholder(tf.float32)
    hidden2_drop = tf.nn.dropout(hidden2, keep_prob)

with tf.name_scope('dense2'):
    w0 = tf.Variable(tf.zeros([num_units2, 10]), name='weight2')
    b0 = tf.Variable(tf.zeros([10]), name = 'dense_bias2')
    p = tf.nn.softmax(tf.matmul(hidden2_drop, w0) + b0)

with tf.name_scope('optimizer'):
    t = tf.placeholder(tf.float32, [None, 10])
    loss = -tf.reduce_sum(t * tf.log(p), name='loss')
    train_step = tf.train.AdamOptimizer(0.0001).minimize(loss)
    correct_prediction = tf.equal(tf.argmax(p, 1), tf.argmax(t, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32), name='accuracy')
    
tf.summary.scalar('loss', loss)
tf.summary.scalar('accuracy', accuracy)
tf.summary.histogram('kernel1', W_conv1)
tf.summary.histogram('kernel2', W_conv2)
tf.summary.histogram('dense1', w2)
tf.summary.histogram('bias1', b2)
tf.summary.histogram('dense2', w0)
tf.summary.histogram('bias2', b0)

<tf.Tensor 'bias2:0' shape=() dtype=string>

In [3]:
sess = tf.Session()
saver = tf.train.Saver(tf.global_variables())

ckpt = tf.train.get_checkpoint_state('./saved_models')
if ckpt and tf.train.checkpoint_exists(ckpt.model_checkpoint_path):
    saver.restore(sess, ckpt.model_checkpoint_path)
else:
    sess.run(tf.global_variables_initializer())

# 스칼라나 히스토그램으로 보여주려는 변수들 모음
merged = tf.summary.merge_all()
# 그래프와 텐서값 저장공간 설정
writer = tf.summary.FileWriter('./logs', sess.graph)
    
print('Start Training')
print('global_step: %s' % tf.train.global_step(sess, global_step))
i = 0
for step in range(5000):
    batch_xs, batch_ts = train_data.next_batch(50)
    feed_dict_train = {x:batch_xs, t:batch_ts, keep_prob:0.5}
    feed_dict_test = {x:eval_data.images, t:eval_data.labels, keep_prob:1.0}
    
    sess.run([train_step, increment_global_step_op],
             feed_dict=feed_dict_train)
    
    if step % 1000 == 0:
        print('Step: %d, ' % sess.run(global_step),
          'loss: %.3f' % sess.run(loss, feed_dict=feed_dict_test),
          'accuracy: %.3f' % sess.run(accuracy, feed_dict=feed_dict_test))
    
    summary = sess.run(merged, feed_dict=feed_dict_train)
    writer.add_summary(summary, global_step=sess.run(global_step))
    
saver.save(sess, './saved_models/tensorboard.ckpt', global_step=global_step)

Start Training
global_step: -1
Step: 0,  loss: 23710.799 accuracy: 0.097
Step: 1000,  loss: 1014.523 accuracy: 0.969
Step: 2000,  loss: 772.287 accuracy: 0.975
Step: 3000,  loss: 602.978 accuracy: 0.980
Step: 4000,  loss: 467.697 accuracy: 0.984


'./saved_models/tensorboard.ckpt-4999'